In [5]:
import sys
sys.path.append('/home/ianlee/opt_ian')

import torch
import rdkit
#import hgraph
import random
import numpy as np
import pyfiglet
import math
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "g1-eno1[0]"
import pickle


from Algorithms.Fittness import Fitness
#from Algorithms.PSO.Arg import arg_builder
from Algorithms.PSO.Create import create_tensor, create_first_particle
from Environments.PvkAdditives.lib.Pvk_Predictor import PvkTransform

#from hgraph import HierVAE, PairVocab
#from torch.utils.data import DataLoader

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from mordred import Calculator, descriptors
import pandas as pd
from rdkit.Chem import PandasTools
#sys.path.append('/home/ianlee/JTVAE/JTVAE/GPU-P3')
from fast_jtnn import *

cutoff_dict = {"crystal_size": [1, None],
            # "crystal_score": [2, None],
        }
bounds = [(176.19463140292243, 0.5665854306219236), 
          (161.68509695291618, 0.4245779984228497), 
          (189.92178705226962, 1.0922707345129656), 
          (169.03341776133132, 0.7437770922756348), 
          (1, 0)]

def natural_log(x):
  return math.log(x)

def load_pvk_data(data_path):
    pvk_feature_list = ['Reagent1 (ul)','Reagent2 (ul)','Reagent3 (ul)','Reagent4 (ul)','lab_code']
    source_data = pd.read_csv(data_path).sample(n=1)
    source_data[pvk_feature_list[:4]] = source_data[pvk_feature_list[:4]].apply(np.log)
    return source_data['SMILES'].iloc[0], source_data[pvk_feature_list].values

if __name__ == "__main__":

    new_bounds = []
    for i, bound in enumerate(bounds):
        if i == 4:
            new_bounds.append(bound)
        else:
            new_bounds.append((natural_log(bound[0]), natural_log(bound[1])))

    #args = arg_builder()

    lg = rdkit.RDLogger.logger()
    lg.setLevel(rdkit.RDLogger.CRITICAL)

    vocab_path = '/home/ianlee/JTVAE/Ian_train/Vocabulary/smi_vocab-2.txt'
    model_path = '/home/ianlee/JTVAE/Ian_train/Train/MODEL-TRAIN-3/model.epoch-26'
    
    # Load vocabulary
    vocab = [x.strip("\r\n ") for x in open(vocab_path)]
    vocab = Vocab(vocab)

    radius = 2.0
    freeze = False
    #vae_model = HierVAE(args).cuda()
    #vae_model.load_state_dict(torch.load(args.model)[0])
    #vae_model.eval()
    
    # Initial Step for VAE
    vae_model = JTNNVAE(vocab, hidden_size=450, latent_size=32, depthT=3, depthG=20)
    vae_model.load_state_dict(torch.load(model_path, map_location='cuda'))
    vae_model.cuda()
    vae_model.eval()

    # target_smiles, initial_process_position = load_pvk_data(args.pvk_data)
    target_smiles, initial_process_position = load_pvk_data('/home/ianlee/opt_ian/Model_Create_and_Results1/Pvk_additives/0_Create_Surrogate_Model/combined_compiledData_wmol.csv')
    print("Original SMILES:", target_smiles)
    latent = vae_model.encode_latent_mean([target_smiles])
    
    #dataset = hgraph.MoleculeDataset([target_smiles], args.vocab, args.atom_vocab, args.batch_size)
    #loader = DataLoader(dataset, batch_size=len(dataset), collate_fn=lambda x:x[0], shuffle=True, num_workers=0)
    #batch = next(iter(loader))
    
    torch.manual_seed(random.randint(0, 2**32 - 1))

    # root_vecs, z_log_var = vae_model.extract_latent_vectors(batch)
    # log_bounds, initial_log_process_position = load_leveler_data(args.leveler_data)
    # print(root_vecs.dtype)
    
    transform = PvkTransform(vae_model, latent_size=32)
    fitness = Fitness(cutoff_dict)
    
    # Create the right feature list
    calc = Calculator(descriptors)

    # Create the right feature list
    print(pyfiglet.figlet_format('Start Create Pool'))
    center_particle = create_first_particle(latent, initial_process_position, bounds, freeze, target_smiles)
    valid_smile_df = pd.DataFrame(columns=["SMILES", "ATSC5v", "AATSC5Z", "MATS8se"])
    valid_tensor = []
    while len(valid_smile_df) < 40:
        total_paticle_position, total_smiles = create_tensor(center_particle, bounds, freeze, radius, 1, transform)
        print(len(valid_smile_df), total_smiles[0])
        esol_data = pd.DataFrame({'SMILES':total_smiles})
        PandasTools.AddMoleculeColumnToFrame(esol_data, smilesCol='SMILES')
        mordred_data = calc.pandas(esol_data['ROMol'])
        mordred_data = mordred_data.dropna(axis='columns')
        numeric_cols = mordred_data.select_dtypes(exclude='number')
        mordred_data.drop(numeric_cols, axis=1, inplace=True)
        print(mordred_data.shape)
        # 檢查每個列是否存在於 DataFrame 中
        columns_exist = all(item in mordred_data.columns for item in ["ATSC5v", "AATSC5Z", "MATS8se",'AATS2p', 'ATSC5Z'])

        if columns_exist:
            # 如果所有列都存在，創建一個新的 DataFrame，只包含 "smile" 列和這三個分子特徵列
            new_data = pd.DataFrame({
                "ATSC5v": mordred_data["ATSC5v"],
                "AATSC5Z": mordred_data["AATSC5Z"],
                "MATS8se": mordred_data["MATS8se"],
                "AATS2p": mordred_data["AATS2p"],
                "ATSC5Z": mordred_data["ATSC5Z"],
                "SMILES": [total_smiles[0]]
            })
            valid_smile_df = valid_smile_df.append(new_data, ignore_index=True)
            valid_tensor.append(total_paticle_position[0][:32])
    valid_tensor = torch.stack(valid_tensor).cuda()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [68]:
for _ in range(2):  # 因為原本已經有1份，所以只需要再複製2次
    valid_smile_df = valid_smile_df.append(valid_smile_df)
    valid_tensor = torch.cat((valid_tensor, valid_tensor), dim=0)

# 將 DataFrame 根據 'smiles' 欄位排序
# valid_smile_df = valid_smile_df.sort_values('SMILES')

# 重設索引，因為排序操作會改變原始的索引
# valid_smile_df = valid_smile_df.reset_index(drop=True)

In [69]:
valid_smile_df.shape

(160, 6)

In [1]:
max_min_dict = {'Reagent1 (ul)': (177, 0), 'Reagent2 (ul)': (162, 0), 'Reagent3 (ul)': (190, 0), 'Reagent4 (ul)': (170, 0), 'lab_code': (0, 1)}

def generate_random_list(length, max_min):
    """
    生成指定長度但數字隨機的list
    Args:
        length: 列表長度
        max_min: 最大最小值元組
    Returns:
        隨機數列表
    """
    max_value = max_min[0]
    min_value = max_min[1]

    # 生成隨機數列表
    random_list = [random.uniform(min_value, max_value) for _ in range(length)]

    return random_list

In [2]:
valid_proc_dict = {}

for key, value in max_min_dict.items():
    valid_proc_dict[key] = generate_random_list(len(valid_smile_df), value)
valid_proc_dict = pd.DataFrame(valid_proc_dict)
valid_proc_dict['lab_code'] = np.where(valid_proc_dict['lab_code'] > 0.5, 1, 0)
# combined_df = pd.concat([valid_smile_df, valid_proc_dict], axis=1)

NameError: name 'valid_smile_df' is not defined

In [72]:
valid_smile_df = valid_smile_df.reset_index(drop=True)
valid_proc_dict = valid_proc_dict.reset_index(drop=True)
combined_df = pd.concat([valid_smile_df, valid_proc_dict], axis=1)

In [ ]:
# 讀取model
import pickle

# 指定模型存儲路徑
size_model_path = '/home/ianlee/opt_ian/Model_Create_and_Results1/Pvk_additives/0_Create_Ground_Truth_Model/pvkadditives/pvk_rfr_size.pkl'
score_model_path = '/home/ianlee/opt_ian/Model_Create_and_Results1/Pvk_additives/0_Create_Ground_Truth_Model/pvkadditives/pvk_rfc_score.pkl'
# 使用 pickle.load() 方法讀取模型
with open(size_model_path, 'rb') as f:
    rf_regressor = pickle.load(f)

with open(score_model_path, 'rb') as f:
    rf_classifier = pickle.load(f)

def pvk_crystal_predict(df: pd.DataFrame, 
                        pvk_rfr: RandomForestRegressor, 
                        pvk_rfc: RandomForestClassifier):
    calc = Calculator(descriptors)
    
    # mored feature creating
    esol_data = pd.DataFrame(df['SMILES'])
    PandasTools.AddMoleculeColumnToFrame(esol_data, smilesCol='SMILES')
    mordred_data = calc.pandas(esol_data['ROMol'])
    mordred_data = mordred_data.dropna(axis='columns')
    numeric_cols = mordred_data.select_dtypes(exclude='number')
    mordred_data.drop(numeric_cols, axis=1, inplace=True)
    print(mordred_data.shape)

    # processing feature creating
    df = df.drop("SMILES", axis=1)

    # combined two data
    try:
        df = df.join(mordred_data)
    except:
        pass

    # scroe predict
    pvk_size_feature_list = ['Reagent1 (ul)', 'Reagent2 (ul)', 'Reagent3 (ul)', 'Reagent4 (ul)', 'lab_code', 'ATSC5v', 'AATSC5Z', 'MATS8se']
    pvk_score_feature_list = ['Reagent1 (ul)', 'Reagent2 (ul)', 'Reagent3 (ul)', 'Reagent4 (ul)', 'AATS2p', 'ATSC5Z']
    df_size = df[pvk_size_feature_list]
    df_score = df[pvk_score_feature_list]

    return pvk_rfr.predict(df_size), pvk_rfc.predict(df_score), df[pvk_size_feature_list], df[pvk_score_feature_list]

/home/ianlee/anaconda3/envs/leveler/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ianlee/anaconda3/envs/leveler/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ianlee/anaconda3/envs/leveler/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using ve

NameError: name 'pd' is not defined

In [74]:
combined_df

,SMILES,ATSC5v,AATSC5Z,MATS8se,AATS2p,ATSC5Z,Reagent1 (ul),Reagent2 (ul),Reagent3 (ul),Reagent4 (ul),lab_code
0,CC(C)CC(NC(=O)C(=O)n1ccnc1)C(=O)O,-128.646778,-0.233157,-0.320968,1.373866,-15.155188,25.188646,63.542088,153.780979,66.790856,1
1,Cc1ccc(-c2ccc(C)cn2)nc1,135.295558,0.026227,0.027596,1.636771,0.970414,148.063673,42.437610,63.259899,108.300024,1
2,CCCC(C)(NCCCCCNCC)C(C)C,-110.048956,-0.118842,0.087382,1.201612,-19.608997,52.045294,27.231983,87.481130,162.549234,0
3,Cc1ccc(-c2ccc(C)cn2)nc1,135.295558,0.026227,0.027596,1.636771,0.970414,24.629891,115.233772,171.457525,151.315992,0
4,CCc1cc(OC2(C)CO2)ccn1,-697.419893,-1.973702,-0.077700,1.321162,-88.816568,133.127865,144.199632,6.961016,136.564334,0
...,...,...,...,...,...,...,...,...,...,...,...
155,O=CC1C=CC(Cc2ccncc2)O1,444.836859,0.909091,-0.217165,1.557061,40.000000,170.106628,1.730305,18.725363,71.585892,0
156,O=C1NC2(CNCCO2)C2=C(C3CN23)C12CNCCO2,-1037.488777,-1.775957,-0.041278,1.393117,-191.803324,31.489069,141.794110,82.698148,144.019402,1
157,CCCc1ccc(-c2ccccn2)nc1,-281.500714,-1.097843,-0.166135,1.587363,-53.794293,92.294979,37.814455,60.188967,127.987407,0
158,Cc1cc(Nc2cccc(CC(=O)CCN)c2)no1,-198.759784,-1.373264,0.109621,1.493842,-87.888889,53.143414,82.655809,79.153080,33.660315,0


In [75]:
pvk_gt_pred = pvk_crystal_predict(combined_df, rf_regressor, rf_classifier)
combined_df['crystal_size'] = list(pvk_gt_pred[0])
combined_df['crystal_score'] = list(pvk_gt_pred[1])
# combined_df.to_csv('cycle0.csv')

100%|██████████| 160/160 [00:02<00:00, 74.90it/s]


(160, 1420)


In [76]:
combined_df

,SMILES,ATSC5v,AATSC5Z,MATS8se,AATS2p,ATSC5Z,Reagent1 (ul),Reagent2 (ul),Reagent3 (ul),Reagent4 (ul),lab_code,crystal_size,crystal_score
0,CC(C)CC(NC(=O)C(=O)n1ccnc1)C(=O)O,-128.646778,-0.233157,-0.320968,1.373866,-15.155188,25.188646,63.542088,153.780979,66.790856,1,0.544706,4.0
1,Cc1ccc(-c2ccc(C)cn2)nc1,135.295558,0.026227,0.027596,1.636771,0.970414,148.063673,42.437610,63.259899,108.300024,1,0.642836,4.0
2,CCCC(C)(NCCCCCNCC)C(C)C,-110.048956,-0.118842,0.087382,1.201612,-19.608997,52.045294,27.231983,87.481130,162.549234,0,0.629595,4.0
3,Cc1ccc(-c2ccc(C)cn2)nc1,135.295558,0.026227,0.027596,1.636771,0.970414,24.629891,115.233772,171.457525,151.315992,0,0.610758,4.0
4,CCc1cc(OC2(C)CO2)ccn1,-697.419893,-1.973702,-0.077700,1.321162,-88.816568,133.127865,144.199632,6.961016,136.564334,0,1.448894,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,O=CC1C=CC(Cc2ccncc2)O1,444.836859,0.909091,-0.217165,1.557061,40.000000,170.106628,1.730305,18.725363,71.585892,0,0.169691,1.0
156,O=C1NC2(CNCCO2)C2=C(C3CN23)C12CNCCO2,-1037.488777,-1.775957,-0.041278,1.393117,-191.803324,31.489069,141.794110,82.698148,144.019402,1,0.759006,4.0
157,CCCc1ccc(-c2ccccn2)nc1,-281.500714,-1.097843,-0.166135,1.587363,-53.794293,92.294979,37.814455,60.188967,127.987407,0,0.810999,4.0
158,Cc1cc(Nc2cccc(CC(=O)CCN)c2)no1,-198.759784,-1.373264,0.109621,1.493842,-87.888889,53.143414,82.655809,79.153080,33.660315,0,1.243534,4.0


In [77]:
valid_proc_values  = valid_proc_dict.values
valid_proc_tensor = torch.from_numpy(valid_proc_values).float().cuda()
valid_proc_tensor

tensor([[ 25.1886,  63.5421, 153.7810,  66.7909,   1.0000],
        [148.0637,  42.4376,  63.2599, 108.3000,   1.0000],
        [ 52.0453,  27.2320,  87.4811, 162.5492,   0.0000],
        [ 24.6299, 115.2338, 171.4575, 151.3160,   0.0000],
        [133.1279, 144.1996,   6.9610, 136.5643,   0.0000],
        [114.0871, 160.3663,  74.9606,  80.4560,   1.0000],
        [153.8964, 150.2458,  29.8901, 145.4846,   0.0000],
        [157.0383,  87.3696, 142.9933,  82.1815,   0.0000],
        [ 69.4923,  65.1347, 131.9304, 163.3794,   0.0000],
        [170.2970,  86.8703,  78.8824,  30.9768,   1.0000],
        [142.4438,  66.0607, 135.6771, 148.6001,   0.0000],
        [ 58.0206,  78.4416,  77.5836,  56.5722,   1.0000],
        [176.8260,  12.0391,  30.3070, 132.2443,   0.0000],
        [163.5732,  62.0804,  35.4912,  33.3156,   1.0000],
        [104.9267,  61.6992, 184.6488,  66.9814,   1.0000],
        [ 13.4742, 147.7420,  47.8233, 126.9280,   1.0000],
        [ 86.8123, 132.6453, 185.8633,  

In [78]:
valid_tensor = torch.cat((valid_tensor, valid_proc_tensor), dim=1)

In [79]:
valid_tensor.shape

torch.Size([160, 37])

# 20240217 Saved

In [80]:
combined_df.to_csv('combined_df.csv')
torch.save(valid_tensor,'valid_tensor.pt')

In [1]:
import pandas as pd
import torch

combined_df_1 = pd.read_csv('combined_df.csv')
valid_tensor_1 = torch.load('valid_tensor.pt')

In [5]:
# from rdkit import Chem

# # 建立一個分子
# mol = Chem.MolFromSmiles('[OH-].[NH4+]')

# # 獲得分子中的所有原子
# atoms = mol.GetAtoms()

# # 對每一個原子進行迭代
# for atom in atoms:
#     # 如果原子是氮原子
#     if atom.GetSymbol() == 'N':
#         # 獲得原子的所有鍵的數量，包括與氫原子的鍵
#         total_degree = atom.GetTotalDegree()
#         print('The nitrogen atom has', total_degree, 'bonds including hydrogen atoms')

The nitrogen atom has 4 bonds including hydrogen atoms
